In [1203]:
import streamlit as st
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import datetime
import time
import millify as my
import calendar

pd.set_option('display.max_columns', None)

st.set_page_config(
    page_title='Personal Finance',
    layout='wide',
    initial_sidebar_state="expanded"
)


In [1204]:
def hold_please():
    msg = st.toast(f'Hi, {name}!')
    time.sleep(1)
    msg.toast('Welcome to my interactive', icon="🔮")
    time.sleep(1)
    msg.toast('Financial Dashboard!', icon="💸",)
    time.sleep(1)
    msg.toast('Done!', icon = "✅")

header = st.container()
title = st.container()
select = st.container()
column1, column2 = st.columns(2)

with column1:
    name = st.text_input(f"What is your name?", on_change=hold_please)

with column2:
    date_input = st.date_input(
        "What is the date you are interested in analyzing?", 
        datetime.date(2022,12,31), format="YYYY-MM-DD"
    )
year_input = date_input.year
month_input = date_input.month
#date_input = datetime.date.today() # future feature

with title: 
    st.subheader('Please Enter Your Name.')
with select: 
    if st.button('Enter'):
        hold_please()
        time.sleep(2)
        name = str(name)+ ("'s") 

    with header: 
        st.title("Samantha's Financial Analysis") 

In [1205]:
@st.cache_data
def load_data(csv):
    df=pd.read_csv(csv)
    return df
    
finance = load_data('data/finance.csv')
finance['Date'] = pd.to_datetime(finance['Date'], format='%Y-%m-%d')
finance_sorted = finance.sort_values('Category')

income = load_data('data/income.csv')
income['Date'] = pd.to_datetime(income['Date'], format='%Y-%m-%d')
#income
#finance

2024-05-06 20:44:51.010 No runtime found, using MemoryCacheStorageManager


In [1206]:
#finance.dtypes
#income.dtypes

In [1207]:
#finance.to_csv('data/finance_clean.csv')

In [1208]:
slider = st.container()
with slider: 
    budget = st.slider(
        "Use the slider to update the financial statistics that follow", 
        min_value=0, max_value=7500, value=50, step=50)
    annual_budget = budget*month_input
    spent_yr = finance.loc[finance["Year"]==year_input]
    spent_mon = spent_yr.loc[spent_yr["Month"]==month_input]

spent_mon_table = spent_mon.copy()
#spent_mon_table

In [1210]:
st.sidebar.title('Filters:')
st.subheader('For use with interactive charts')

expense_type = st.sidebar.multiselect(
    "Category:",
    options=finance_sorted["Category"].unique(),
    default=finance_sorted["Category"].unique()
)
month = st.sidebar.multiselect(
    "Month:",
    options=finance["Month"].unique(),
    default=finance["Month"].unique()
)
year = st.sidebar.multiselect(
    "Year:",
    options=finance["Year"].unique(),
    default=finance["Year"].unique()
)

In [1211]:
#spent_mon_table.dtypes
#spent_mon_query = spent_mon.copy()

In [1212]:
finance_query = finance.query("Year==@year & Month==@month & Category==@expense_type")

In [1213]:
spent_total_mon = spent_yr.loc[spent_yr['Month']==month_input]['Debit'].sum()
spent_mon_var = ((spent_total_mon-budget)*(-1)).round(2)

spent_total_yr = spent_yr['Debit'].sum()
yr_var = ((spent_total_yr-annual_budget)*(-1)).round(2)

In [1214]:
#print(income.columns)

In [1215]:
monthly_income = income.loc[income['Month'] == month_input, 'Income'].sum().round(2)
monthly_income_var = ((monthly_income - spent_total_mon) * (1)).round(2)

yearly_income = income.loc[income['Year'] == year_input, 'Income'].sum().round(2)
yearly_income_var = ((yearly_income - spent_total_yr) * (-1)).round(2)

In [1216]:
spent_mon_query_group = spent_mon_query.groupby(['Month','Year','Category'])['Debit'].sum().reset_index()
spent_mon_query_group.columns = ['Month','Year','Category','Total Debits']
spent_mon_query_sort = spent_mon_query_group.sort_values('Total Debits', ascending=False).head(12).reset_index(drop=True)

In [1217]:
spent_mon_fig = px.bar(spent_mon_query_sort, x='Category', y='Total Debits', color_discrete_sequence=px.colors.qualitative.Prism)

spent_mon_fig.update_traces(hovertemplate="<b>Category: </b> %{x}<br><b>Total: $</b> %{y}<br>")
spent_mon_fig

In [1218]:

fig = go.Figure(data=go.Table(
    header=dict(
        values=list(spent_mon_table[['Date', 'Account', 'Debit', 'Credit', 'Category']].columns),
        fill_color='#6b4ea7',
        font=dict(color='#FFFFFF'),
        align='center'
    ),
    cells=dict(
        values=[
            spent_mon_table.Date.dt.strftime('%m-%d-%Y'), 
            spent_mon_table.Account, spent_mon_table.Debit, 
            spent_mon_table.Credit, spent_mon_table.Category
            ],
        align='left'
    )
))

#fig.update_layout()
fig

In [1219]:
spent_past_table = finance.copy()
#spent_past_table

In [1220]:
spent_mon_query_group = spent_mon_query.groupby(['Month','Year','Category'])['Debit'].sum().reset_index()
spent_mon_query_group.columns = ['Month','Year','Category','Total Debits']
spent_mon_query_sort = spent_mon_query_group.sort_values('Total Debits', ascending=False).head(12).reset_index(drop=True)


In [1221]:
spent_yr
#finance_query = finance.query("Year==@year & Month==@month & Category==@expense_type")

,Date,Account,Debit,Credit,Category,Month,Year
673,2022-01-03,CHK,0.00,130.0,Entertainment,1,2022
674,2022-01-03,CHK,1500.00,0.0,Savings,1,2022
675,2022-01-03,CC BofA,49.95,0.0,Insurance,1,2022
676,2022-01-03,CC BofA,83.80,0.0,Self Care,1,2022
677,2022-01-03,CC BofA,50.73,0.0,Food,1,2022
...,...,...,...,...,...,...,...
1362,2022-12-29,CC BofA,5.00,0.0,Pet,12,2022
1363,2022-12-30,CHK,82.23,0.0,Pet,12,2022
1364,2022-12-30,CC BofA,39.14,0.0,Food,12,2022
1365,2022-12-30,CC BofA,32.93,0.0,Food,12,2022


In [1222]:
spent_past_query=spent_yr.copy()
spent_past_query_group = spent_past_query.groupby(['Date', 'Account', 'Category'])[['Debit', 'Credit']].sum().round().reset_index()
spent_past_query_group = spent_past_query_group.rename(columns={'Debit': 'Total Debits', 'Credit': 'Total Credits'})
spent_past_query_ordered = spent_past_query_group.sort_values('Date', ascending=False)
spent_past_query_ordered.columns = ['Date', 'Account', 'Category', 'Debit', 'Credit']
spent_past_query_sort = spent_past_query_ordered.sort_values('Date', ascending=False).reset_index(drop=True)

In [1223]:
#with year: fix indentation
spent_past_table = finance.copy()
fig = go.Figure(data=go.Table(
        header=dict(
        values=list(spent_past_table[['Date', 'Account', 'Debit', 'Credit', 'Category']].columns),
        align='center',
        fill_color='#5F4690',
        font=dict(color='#FFFFFF')
        ),
        cells=dict(
        values=[
            spent_past_table.Date.dt.strftime('%m-%d-%Y'), 
            spent_past_table.Account, spent_past_table.Debit, 
            spent_past_table.Credit, spent_past_table.Category
            ],
        align='left'
        )
    ))
fig.update_layout(margin= dict(t=0,l=0,r=0,b=0))
#st.plotly_chart(fig, use_container_width=True)
fig

In [1224]:
#spent_past_table = finance.copy()
fig = go.Figure(data=go.Table(
    header=dict(
        values=list(spent_past_query_sort[['Date', 'Account', 'Debit', 'Credit', 'Category']].columns),
        fill_color='#6b4ea7',
        font=dict(color='#FFFFFF'),
        align='center'
    ),
    cells=dict(
        values=[
            spent_past_query_sort.Date.dt.strftime('%m-%d-%Y'), 
            spent_past_query_sort.Account, 
            spent_past_query_sort.Debit, 
            spent_past_query_sort.Credit, 
            spent_past_query_sort.Category
            ],
        align='left',
        fill_color='#E5ECF6'
    )
))
fig.update_layout(margin= dict(t=0,l=0,r=0,b=0))
st.plotly_chart(fig, use_container_width=True)

DeltaGenerator()

In [1225]:
finance_query = finance.query("Year==@year & Month==@month & Category==@expense_type")

In [1226]:
finance_query

,Date,Account,Debit,Credit,Category,Month,Year
0,2021-01-02,CC - BofA,0.00,127.79,Shopping,1,2021
1,2021-01-02,CC - AMEX,18.29,0.00,Entertainment,1,2021
2,2021-01-04,CHK,0.00,450.88,Rent,1,2021
3,2021-01-04,CHK,0.00,600.00,Taxes,1,2021
4,2021-01-04,CHK,0.00,19.30,Utility,1,2021
...,...,...,...,...,...,...,...
1362,2022-12-29,CC BofA,5.00,0.00,Pet,12,2022
1363,2022-12-30,CHK,82.23,0.00,Pet,12,2022
1364,2022-12-30,CC BofA,39.14,0.00,Food,12,2022
1365,2022-12-30,CC BofA,32.93,0.00,Food,12,2022


In [1227]:
#income

In [1274]:
spend_by_year = finance_query.groupby(by=["Year"])[["Debit"]].sum().round(2).reset_index()
credit_by_year = finance_query.groupby(by=["Year"])[["Credit"]].sum().round(2).reset_index()
credit_by_year = credit_by_year.rename(columns={'Credit': 'Income'})
income_table = income.groupby(by=["Year"])[["Income"]].sum().round(2).reset_index()

In [1275]:
combined = pd.merge(credit_by_year,income_table, how='inner', on='Year')
combined['Income'] = combined['Income_x'] + combined['Income_y']
combined.drop(['Income_x', 'Income_y'], axis=1, inplace=True)
combined = combined.merge(spend_by_year, how='left', on='Year')
combined = combined.rename(columns = {'Debit':'Debits'})
combined_melted = combined.melt(id_vars='Year', var_name='Type', value_name='Amount')

In [1276]:
fig = px.bar(
    combined_melted, 
    x='Year', 
    y='Amount', 
    color='Type', 
    barmode='group', 
    color_discrete_sequence=['darkcyan', 'orange'])
fig.update_layout(
                  xaxis_title='Year',
                  yaxis_title='Amount',
                  legend_title='Type')
fig.update_traces(
    hovertemplate="<b>Year: </b> %{x}<br><b>Amount: $</b> %{y}<br>"
    )

fig.show()

In [1262]:
#finance_query

In [1264]:
time_series = st.container()
spend_by_year=finance.copy()
credit_by_year=finance.copy()
finance_query = finance.query('Year == @year & Category == @expense_type')
with time_series:
    st.subheader('Spending Over Time')
    spend_by_year = finance_query.groupby(by=['Year'])[['Debit']].sum()
    credit_by_year = finance_query.groupby(by=['Year'])[['Credit']].sum()
    combined = pd.merge(spend_by_year, credit_by_year,left_index=True, right_index=True)
    combined = combined.stack().reset_index()
    combined.columns = ['Year', 'Category', 'Amount']

In [1265]:
total_debit = finance.groupby(['Date'])['Debit'].sum().reset_index(name='Debits')
total_credit = finance.groupby(['Date'])['Credit'].sum().reset_index(name='Credits')
dual = pd.merge(total_debit, total_credit, on='Date', how='outer')
#dual.dtypes

In [1266]:
dual_cumulative = dual[['Debits', 'Credits']].cumsum()
fig1 = px.area(
    dual_cumulative, x=dual['Date'], 
    y=dual_cumulative.columns,
    color_discrete_map={'Debits': 'darkcyan', 'Credits': 'orange'}
    )

custom_text = ['Type: {}'.format(col) for col in dual_cumulative.columns]

hover_template = (
    '<b>Type:</b>'
    '<b>Date:</b> %{x}<br>'
    '<b>Cumulative Amount:</b> $%{y:,.0f}'
)

fig1.update_traces(
    mode='lines+markers',
    hovertemplate=hover_template,
    customdata=dual_cumulative.columns
)

fig1.update_layout(
    legend_title='Transaction Type',
    xaxis_title="Date",
    yaxis_title="Cumulative Amount"
)

fig1.update_xaxes(title_text="Date")
fig1.update_yaxes(title_text="Cumulative Amount")

fig1.show()


In [1297]:
spend_by_year = finance_query.groupby(by=['Date'])[["Debit"]].sum().round(2).reset_index()
spend_by_year = spend_by_year.rename(columns={'Debit': 'Debits'})
credit_by_year = finance_query.groupby(by=['Date'])[["Credit"]].sum().round(2).reset_index()
credit_by_year = credit_by_year.rename(columns={'Credit': 'Income'})
income_table = income.groupby(by=['Date'])[["Income"]].sum().round(2).reset_index()

#spend_by_year
#credit_by_year
#income_table

In [1309]:
dual = pd.merge(credit_by_year,income_table, how='left', on='Date')
dual['Income'] = dual['Income_x'] + dual['Income_y']
dual.drop(['Income_x', 'Income_y'], axis=1, inplace=True)
dual = dual.merge(spend_by_year, how='left', on='Date')
dual['Income'] = dual['Income'].fillna(0.00).astype(float)


In [1310]:
dual_cumulative = dual[['Debits', 'Income']].cumsum()
fig1 = px.area(
    dual_cumulative, x=dual['Date'], 
    y=dual_cumulative.columns,
    color_discrete_map={'Credits': 'darkcyan', 'Debits': 'orange'}
    )

custom_text = ['Type: {}'.format(col) for col in dual_cumulative.columns]

hover_template = (
    '<b>Type:</b>'
    '<b>Date:</b> %{x}<br>'
    '<b>Cumulative Amount:</b> $%{y:,.0f}'
)

fig1.update_traces(
    mode='lines+markers',
    hovertemplate=hover_template,
    customdata=dual_cumulative.columns
)

fig1.update_layout(
    legend_title='Transaction Type',
    xaxis_title="Date",
    yaxis_title="Cumulative Amount"
)

fig1.update_xaxes(title_text="Date")
fig1.update_yaxes(title_text="Cumulative Amount")

fig1.show()
